In [1]:
!pip install gigachain -q
!pip install sentence-transformers -q
!pip install faiss-cpu
!pip install gigachain gigachain_community
!pip install gigachain-core
!pip install pypdf
#!pip install sacrebleu Вроде в итоге не надо, но использовалось для первой попытки BLEU
!pip install nltk
!pip install rouge-score
#!pip install evaluate Где то виде, что надо для BERTScore, пока решил отложить
!pip install transformers sentence-transformers datasets #Возможно sentence-transformers лишний, т.к. уже установлен

In [2]:
from getpass import getpass
from langchain.chat_models import GigaChat

In [3]:
gigachat_api_key = getpass(prompt='Введите API ключ от GigaChat')

Введите API ключ от GigaChat··········


In [4]:
# Инициализируем языковую модель GigaChat
# verify_ssl_certs=False – без использования сертификатов Минцифры
llm = GigaChat(credentials=gigachat_api_key, verify_ssl_certs=False, temperature=0.2)

In [5]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

In [ ]:
# Список путей к документам
document_paths = [
    "FSB196.pdf",
    "FSB281.pdf",
    "FSB282.pdf",
    "FSB366.pdf",
    "FSB367.pdf",
    "FSB368.pdf",
    "FSTEK235.pdf",
    "FSTEK236.pdf",
    "FSTEK239.pdf",
    "FSTEK75.pdf",
    "FZ187.pdf",
    "PP127.pdf",
    "PP1272.pdf",
    "PP1478.pdf",
    "PP162.pdf",
    "PP1912.pdf",
    "PP2461.pdf",
    "PP743.pdf",
    "YK166.pdf",
    "YK250.pdf"
]

In [ ]:
# Список путей к документам с ответами на вопросы
document_paths_new = [
    "1).pdf",
    "2).pdf",
    "3).pdf",
    "4).pdf",
    "5).pdf",
    "6).pdf",
    "7).pdf",
    "8).pdf",
    "9).pdf",
    "10).pdf",
    "11).pdf",
    "12).pdf",
    "13).pdf",
    "14).pdf",
    "15).pdf",
    "16).pdf",
    "17).pdf",
    "18).pdf",
    "19).pdf",
    "20).pdf",
    "21).pdf",
    "22).pdf",
    "23).pdf",
    "24).pdf",
    "25).pdf",
    "26).pdf",
    "27).pdf",
    "28).pdf",
    "29).pdf",
    "30).pdf",
    "31).pdf",
#   "32).pdf",
    "33).pdf",
    "34).pdf",
    "35).pdf",
    "36).pdf",
    "37).pdf",
    "38).pdf",
    "39).pdf",
    "40).pdf",
    "41).pdf",
    "42).pdf"
]

In [26]:
# Список путей к документам с ответами на вопросы
document_paths_new = [
    "8).pdf",
    "18).pdf",
    "24).pdf",
    "25).pdf",
    "28).pdf",
    "38).pdf"
]

In [27]:
# Загрузка всех документов
from langchain_community.document_loaders import PyPDFLoader
documents = []
for path in document_paths_new:
    loader = PyPDFLoader(path)
    documents.extend(loader.load())

In [28]:
# Разделение текста на части
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
split_docs = text_splitter.split_documents(documents)
print(f"Total documents: {len(split_docs)}")

Total documents: 521


In [29]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [30]:
# Если у вас нет видеокарты, укажите 'device': 'cpu'
hf_embeddings_model = HuggingFaceEmbeddings(
        model_name="cointegrated/LaBSE-en-ru"
    )

# Создаем FAISS индекс (базу векторов) с полученными эмбеддингами
db = FAISS.from_documents(split_docs, hf_embeddings_model)

##QnA цепочка

In [31]:
from langchain.chains import RetrievalQA

In [32]:
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

##Создание списков вопросов и ответов данных и сгенерированных LLM

In [33]:
import pandas as pd

In [34]:
questions = pd.read_excel("q_data.xlsx", nrows=3).Question
answers = pd.read_excel("q_data.xlsx", nrows=3).Answer

In [17]:
questions

0    СБ ЗОКИИ: какие ограничения существуют по допу...
1    Серия вебинаров: моделирование угроз должно вы...
2    СБ ЗОКИИ: допускается ли использовать функции ...
Name: Question, dtype: object

In [18]:
answers

0    Ответ: прямые ограничения по допуску иностранн...
1    Ответ: первичное моделирование угроз безопасно...
2    Ответ: да, допускается, однако в отношении исп...
Name: Answer, dtype: object

In [35]:
message = []
for e in questions:
    message.append(qa_chain({"query": e})['result'])

In [36]:
message

['В значимом объекте не допускается наличие удаленного доступа к программным и программно-аппаратным средствам, в том числе средствам защиты информации, для обновления или управления со стороны лиц, не являющихся работниками субъекта критической информационной инфраструктуры.',
 'Моделирование угроз должно выполняться до разработки технического задания на систему обеспечения информационной безопасности (СОИБ).',
 'Не знаю']

##BLEUMetric

Вторая попытка реализации (Удачная)

In [37]:
#Импортируем функцию вычисления BLEU
from nltk.translate.bleu_score import corpus_bleu

#Оборачиваем ответы в необходимую форму для вычесления метрики BLEU
references = [[[ref]] for ref in message]
translations = [[trans] for trans in answers]

#Вычисляем BLEU Score
bleu_score_corpus = corpus_bleu(references, translations)
print("Corpus BLEU Score: ", bleu_score_corpus)

Corpus BLEU Score:  0


In [38]:
#Откуда взят код
references = [['I', 'love', 'eating', 'ice', 'cream'], ['He', 'enjoys', 'eating', 'cake']]
translations = [['I', 'love', 'eating', 'ice', 'cream'], ['He', 'likes', 'to', 'eat', 'cake']]

# Create a list of reference lists
references_list = [[ref] for ref in references]

# Calculate BLEU score for the entire corpus
bleu_score_corpus = corpus_bleu(references_list, translations)
print("Corpus BLEU Score: ", bleu_score_corpus)

Corpus BLEU Score:  0.5438786529686386


##Первая попытка реализации (что то не удалось)

In [ ]:
class BLEUMetric(object):
    '''
        Входные данные:
         * refs:list(str): - список обработанных эталонных ответов.
         * resps:list(str): - список обработанных ответов LLM.
        Return:
         * bleu:list(float): - список метрик BLEU для каждой пары
                                    эталонный ответ и ответ LLM.
         * ref_len:list(int): - длина эталонного ответа, полученна при расчете метрики BLEU.
         * resp_len:list(int): - длина ответа LLM, полученна при расчете метрики BLEU.
    '''

    def __init__(self, refs: list, resps:list):
        self.refs = refs
        self.resps = resps

    def evaluate_metric(self):
        bleu = []
        ref_len = []
        resp_len = []

        for ref, res in tqdm(zip(self.refs, self.resps)):
            scores = sacrebleu.sentence_bleu(res, [ref])
            bleu.append(round(scores.score/100, 3))
            ref_len.append(scores.ref_len)
            resp_len.append(scores.sys_len)

        return bleu, ref_len, resp_len

In [ ]:
from tqdm import tqdm
import sacrebleu

In [ ]:
bm = BLEUMetric(answers, message).evaluate_metric()
df = pd.read_csv("fileq.csv", sep = ';')
df['bleu'] = bm[0]
df['ref_len'] = bm[1]
df['resp_len'] = bm[2]
df

118it [00:00, 1147.78it/s]


ValueError: Length of values (118) does not match length of index (1007)

In [ ]:
test = pd.read_excel(PATH)

NameError: name 'PATH' is not defined

In [ ]:
table = self.df.groupby(['model_name'], as_index=False)[self.df.columns[7:]].agg(['mean']).round(3).reset_index()
        table.columns = table.columns.droplevel(1)
        table.columns = table.columns

In [ ]:
'''       * refs:list(str): = answers - список обработанных эталонных ответов.
          * resps:list(str): = message - список обработанных ответов LLM.
'''

bleu = []
ref_len = []
resp_len = []

for ref, res in tqdm(zip(answers, message)):
    scores = sacrebleu.sentence_bleu(res, [ref])
    bleu.append(round(scores.score/100, 3))
    ref_len.append(scores.ref_len)
    resp_len.append(scores.sys_len)
    print(bleu, ref_len , resp_len)
bm = object(bleu, ref_len , resp_len)
bm.evaluate_metric()

df = []
df['bleu'] = bm[0]
df['ref_len'] = bm[1]
df['resp_len'] = bm[2]
df

6it [00:00, 184.73it/s]

[0.0] [318] [14]
[0.0, 0.0] [318, 461] [14, 17]
[0.0, 0.0, 0.0] [318, 461, 236] [14, 17, 2]
[0.0, 0.0, 0.0, 0.0] [318, 461, 236, 217] [14, 17, 2, 2]
[0.0, 0.0, 0.0, 0.0, 0.0] [318, 461, 236, 217, 385] [14, 17, 2, 2, 2]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0] [318, 461, 236, 217, 385, 413] [14, 17, 2, 2, 2, 2]


([0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [318, 461, 236, 217, 385, 413],
 [14, 17, 2, 2, 2, 2])

BLEU из другого места


In [ ]:
!pip install bleu

  Preparing metadata (setup.py) ... done
  Created wheel for bleu: filename=bleu-0.3-py3-none-any.whl size=5781 sha256=3beae6428d7f2a86339809a844f439e33e22e50cb8e22b6a39129632d9786fc5
  Stored in directory: /root/.cache/pip/wheels/c6/d8/d1/009abe01b8b2c6a14c62d197b510b3cc1076014c22d712c5ce
Successfully built bleu


In [ ]:
from bleu import *

In [ ]:
predictions = ["hello there general kenobi", "foo bar foobar"]
references = [
    ["hello there general kenobi", "hello there !"],
    ["foo bar foobar"]
]
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
print(results)

NameError: name 'evaluate' is not defined

Др

##ROUGE

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
r_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rougeL, другой компонент оценки ROUGE, вычисляет самую длинную общую подпоследовательность (LCS) между системными и справочными сводками. В отличие от N-грамм, LCS измеряет максимальную последовательность слов (не обязательно смежных), которые появляются в обоих сводках. Он предлагает более гибкую меру сходства и помогает фиксировать общую информацию, выходящую за рамки строгих дословных совпадений.

Не разобрался, какой из rouge1, rouge2 является N, а какой S

ROUGE-N - это компонент оценки ROUGE, который количественно определяет перекрытие N-граммов, непрерывных последовательностей из N элементов (обычно слов или символов), между сгенерированной системой сводкой и справочной сводкой. Он дает представление о точности и повторяемости выходных данных системы, учитывая совпадающие N-граммовые последовательности.

ROUGE-S фокусируется на пропуск-биграммах. Пропуск-биграмма - это пара слов в предложении, которая допускает пробелы или слова между ними. Этот компонент определяет перекрытие пропусков и биграмм между системным и справочным резюме, позволяя оценить сходство структуры на уровне предложений. Он может фиксировать перефразирующие связи между предложениями и давать представление о способности системы передавать информацию с гибким порядком слов.

In [ ]:
#Код, на который оперался
candidate_summary = "the cat was found under the bed"
reference_summary = "the cat was under the bed"
r_scores = r_scorer.score(reference_summary, candidate_summary)
for key in r_scores:
    print(f'{key}: {r_scores[key]}')

rouge1: Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923)
rouge2: Score(precision=0.6666666666666666, recall=0.8, fmeasure=0.7272727272727272)
rougeL: Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923)


In [ ]:
r_scores['rouge1']

Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923)

In [ ]:
df = pd.DataFrame(columns=['Question', 'Answer', 'LLM_Answer', 'rouge1', 'rouge2', 'rougeL'])

,Question,Answer,LLM_Answer,rouge1,rouge2,rougeL


In [ ]:
for e in range(len(questions)):
  candidate_summary = message[e]
  reference_summary = answers[e]
  r_scores = r_scorer.score(reference_summary, candidate_summary)
  new_row = {'Question': questions[e], 'Answer': answers[e], 'LLM_Answer': message[e], 'rouge1': r_scores['rouge1'], 'rouge2': r_scores['rouge2'], 'rougeL': r_scores['rougeL']}
  df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

In [ ]:
df

,Question,Answer,LLM_Answer,rouge1,rouge2,rougeL
0,СБ ЗОКИИ: какие ограничения существуют по допу...,Ответ: прямые ограничения по допуску иностранн...,Иностранный гражданин не вправе осуществлять э...,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)","(0, 0, 0)"
1,Серия вебинаров: моделирование угроз должно вы...,Ответ: первичное моделирование угроз безопасно...,Моделирование угроз должно выполняться до разр...,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)","(0, 0, 0)"
2,СБ ЗОКИИ: допускается ли использовать функции ...,"Ответ: да, допускается, однако в отношении исп...","Нет, использование функций защиты прикладного ...","(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)"
3,СБ ЗОКИИ: допускается ли использовать ПО с отк...,Ответ: использование программного обеспечения ...,Не знаю,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)","(0, 0, 0)"
4,Cерия вебинаров: в составе ЗОКИИ имеется котел...,Ответ: меры по обеспечению безопасности значим...,Не знаю,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)","(0, 0, 0)"
...,...,...,...,...,...,...
113,ГосСОПКА: в какие сроки нужно сообщить об инци...,Ответ: согласно п. 4приказа ФСБ России от 19.0...,Не знаю,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)","(0, 0, 0)"
114,Персонал: имеется ли возможность (нет ли юриди...,Ответ: в соответствии сприказом ФСТЭК России о...,Не знаю,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)","(0, 0, 0)"
115,Персонал: какие курсы профессиональной перепод...,Ответ:приказом ФСТЭК России от 27.03.2019 № 64...,"Сотрудникам, ответственным за безопасность ЗОК...","(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)","(0, 0, 0)"
116,Персонал: требуется ли круглосуточное дежурств...,Ответ: не требуется.Требование об осуществлени...,Не знаю,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)","(0, 0, 0)"


##Семантическая похожесть на основе Sentence Transformers

In [ ]:
class SemanticSimilarityMetric(object):
    '''
        Входные данные:
         * refs:list(str): - список обработанных эталонных ответов.
         * resps:list(str): - список обработанных ответов LLM.
        Return:
         * sem_sim:list(float): - список метрик косинусное сходство для каждой пары
                                эталонный ответ и ответ LLM.
    '''
    def __init__(self, refs: list, resps:list):
        self.refs = refs
        self.resps = resps
        self.model = SentenceTransformer(MODEL)

    def evaluate_metric(self):
        sem_sim = []

        for ref, res in tqdm(zip(self.refs, self.resps)):
            #Построение эмбеддингов с помощью предобученной модели all-MiniLM-L6-v2
            reference_embedding = self.model.encode(ref, convert_to_tensor = True)
            response_embedding = self.model.encode(res, convert_to_tensor = True)
            #Расчет косинусного сходства
            cosine_score = util.cos_sim(reference_embedding, response_embedding)
            sem_sim.append(round(float(cosine_score), 3))

        return sem_sim

In [ ]:
sem_sim = []
for ref, res in tqdm(zip(self.refs, self.resps)):


In [ ]:
!pip install transformers sentence-transformers datasets

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_score = cosine_similarity(embeddings1, embeddings2)

In [ ]:
# Если у вас нет видеокарты, укажите 'device': 'cpu'
hf_embeddings_model = HuggingFaceEmbeddings(
        model_name="cointegrated/LaBSE-en-ru"
    )

# Создаем FAISS индекс (базу векторов) с полученными эмбеддингами
db = FAISS.from_documents(split_docs, hf_embeddings_model)

In [ ]:
print(hf_embeddings_model)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 768, 'out_features': 768, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
  (3): Normalize()
) model_name='cointegrated/LaBSE-en-ru' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False
